In [ ]:
# prompt: mount

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# ▶  Build 6-hour-history features for 2018-2022 training set
import pandas as pd
from pathlib import Path

SRC  = Path("/content/drive/MyDrive/PatchedOutput/meteorological_imputed.csv")
DST  = Path("/content/drive/MyDrive/meteo_2018-22_last6h.csv")

vars = ["air_temp_C",
        "dew_point_C",
        "relative_humidity_percent",
        "wind_speed_m_s",
        "precipitation_in"]

# 1) load full hourly meteo
df = pd.read_csv(SRC, parse_dates=['datetime'])
df["date"] = df["datetime"].dt.date          # plain date

rows = []
for day, grp in df.groupby("date"):
    t10   = pd.Timestamp(day).replace(hour=10)
    mask  = (grp["datetime"] > t10 - pd.Timedelta(hours=6)) & (grp["datetime"] <= t10)
    last6 = grp.loc[mask].sort_values("datetime")

    if len(last6) != 6:         # skip days with missing hours
        continue

    row = {"date": day}
    for h, (_, r) in enumerate(last6.iterrows()):
        lag = 5 - h             # 5h ago … 0h
        for v in vars:
            row[f"{v}_t-{lag}h"] = r[v]
    rows.append(row)

feat_df = pd.DataFrame(rows)
feat_df.to_csv(DST, index=False)
print(f"✅ wrote {len(feat_df)} daily rows → {DST}")


✅ wrote 1149 daily rows → /content/drive/MyDrive/meteo_2018-22_last6h.csv


In [ ]:
import pandas as pd

PATCH_CSV = "/content/drive/MyDrive/PatchedOutput/patch_with_meteo.csv"  # old 10 AM file
NEW_METEO = "/content/drive/MyDrive/meteo_2018-22_last6h.csv"
OUT_CSV   = "/content/drive/MyDrive/patch_with_meteo_last6h.csv"

patch_df = pd.read_csv(PATCH_CSV, parse_dates=['date'])
patch_df['date'] = patch_df['date'].dt.date        # ensure plain date

meteo_df = pd.read_csv(NEW_METEO, parse_dates=['date'])
meteo_df['date'] = meteo_df['date'].dt.date

merged = patch_df.merge(meteo_df, on='date', how='inner')
print("kept", len(merged), "patch rows after merge")
merged.to_csv(OUT_CSV, index=False)
print("✅ saved", OUT_CSV)


kept 11474 patch rows after merge
✅ saved /content/drive/MyDrive/patch_with_meteo_last6h.csv


In [ ]:
# Cell 1 ▶ Install required packages
!pip install timm torchvision tifffile imagecodecs --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 MB 11.6 MB/s eta 0:00:00


In [ ]:
# Cell 2 ▶ Consolidated imports
import os
import math
import time

import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split

import timm
from torchvision import transforms, models

from tifffile import imread

In [ ]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 74.9 MB/s eta 0:00:00


In [ ]:
# Cell 1 ▶ Dataset (fix sequence lags 5→0)
import os
import rasterio
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset
from torchvision import transforms

class LSTDataset(Dataset):
    def __init__(self, df, patches_dir, base_vars, k=6):
        self.df        = df.reset_index(drop=True)
        self.dir       = patches_dir
        self.base_vars = base_vars  # list of 5 var names
        self.k         = k         # history length
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485,0.456,0.406],
                std =[0.229,0.224,0.225]
            ),
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row  = self.df.loc[idx]
        path = os.path.join(self.dir, row['filename'])

        # read TIFF via rasterio
        with rasterio.open(path) as src:
            arr = src.read().astype(np.float32)  # (bands, H, W)

        # image input
        img_np = arr[[1,2,3]].transpose(1,2,0).astype(np.uint8)
        img    = self.transform(img_np)       # [3,224,224]

        # target LST at 224×224
        tar = torch.tensor(arr[0], dtype=torch.float32).unsqueeze(0)
        tar = F.interpolate(
            tar.unsqueeze(0), size=(224,224),
            mode='bilinear', align_corners=False
        ).squeeze(0)                         # [1,224,224]

        # build weather sequence matching t-5h…t-0h
        seq = []
        for lag in range(self.k-1, -1, -1):  # 5,4,3,2,1,0
            key = f"{self.base_vars[0]}_t-{lag}h"  # test with air_temp_C
            # ensure the key exists
            if key not in row:
                raise KeyError(f"Expected column '{key}' in DataFrame")
            seq.append([row[f"{v}_t-{lag}h"] for v in self.base_vars])
        weather_seq = torch.tensor(seq, dtype=torch.float32)  # [6,5]

        return img, weather_seq, tar


In [ ]:
# Cell 2 ▶ Weather sequence specs
# These must match exactly how you flattened your CSV:
base_vars = [
    "air_temp_C",
    "dew_point_C",
    "relative_humidity_percent",
    "wind_speed_m_s",
    "precipitation_in",
]
k = 6   # we expect columns _t-5h … _t-0h for each of these 5 vars


In [ ]:
# Cell 3 ▶ Load merged patch-to-meteo CSV & build DataLoaders
import os
import pandas as pd
from torch.utils.data import random_split, DataLoader

MERGED_CSV = "/content/drive/MyDrive/patch_with_meteo_last6h.csv"
PATCH_DIR  = "/content/drive/MyDrive/PatchedOutput_Cleaned"

# 1) Load & rename filename column if needed
df = pd.read_csv(MERGED_CSV, parse_dates=['date'])
if 'filename' not in df.columns:
    if 'patch_filename' in df.columns:
        df = df.rename(columns={'patch_filename':'filename'})
    else:
        raise KeyError("Cannot find 'filename' or 'patch_filename' in CSV")
df['date'] = df['date'].dt.date

# 2) Instantiate your custom Dataset
dataset = LSTDataset(df, PATCH_DIR, base_vars, k=k)

# 3) Train/Val split
n_train = int(0.8 * len(dataset))
train_ds, val_ds = random_split(dataset, [n_train, len(dataset)-n_train])

# 4) DataLoaders
train_loader = DataLoader(train_ds, batch_size=4, shuffle=True,
                          num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=4, shuffle=False,
                          num_workers=2, pin_memory=True)

print(f"✅ Loaded {len(dataset)} samples → train={len(train_ds)}, val={len(val_ds)}")


✅ Loaded 11474 samples → train=9179, val=2295


In [ ]:
import math, torch.nn as nn, timm, torch

class ViT_GRU_LST(nn.Module):
    def __init__(self,
                 k_hours=6,         # sequence length
                 feature_dim=5,     # per-hour vars
                 hidden_dim=768,
                 vit_name="vit_base_patch16_224",
                 num_layers=2, num_heads=8):
        super().__init__()
        self.vit = timm.create_model(vit_name, pretrained=True, num_classes=0)
        for p in self.vit.parameters(): p.requires_grad = False

        # GRU encodes [B,6,5] → hidden 768
        self.gru = nn.GRU(feature_dim, hidden_dim, batch_first=True)

        enc = nn.TransformerEncoderLayer(
            d_model=hidden_dim, nhead=num_heads,
            dim_feedforward=hidden_dim*4, dropout=0.1)
        self.transformer = nn.TransformerEncoder(enc, num_layers)

        # decoder 14→28→56→112→224
        self.deconv = nn.Sequential(
            nn.ConvTranspose2d(hidden_dim, hidden_dim//2, 2, 2),
            nn.BatchNorm2d(hidden_dim//2), nn.ReLU(True),
            nn.ConvTranspose2d(hidden_dim//2, hidden_dim//4, 2, 2),
            nn.BatchNorm2d(hidden_dim//4), nn.ReLU(True),
            nn.ConvTranspose2d(hidden_dim//4, hidden_dim//8, 2, 2),
            nn.BatchNorm2d(hidden_dim//8), nn.ReLU(True),
            nn.ConvTranspose2d(hidden_dim//8, 1, 2, 2)
        )

    def forward(self, x, w_seq):
        # ViT features
        feats   = self.vit.forward_features(x)          # [B,197,768]
        cls_tok = feats[:, :1]
        patch_t = feats[:, 1:]                          # 196 tokens

        # GRU weather → last hidden as token
        _, h_n  = self.gru(w_seq)                       # [1,B,768]
        w_tok   = h_n.transpose(0,1)                    # [B,1,768]

        tokens  = torch.cat([patch_t, w_tok, cls_tok], 1)  # [B,198,768]
        t = self.transformer(tokens.permute(1,0,2)).permute(1,0,2)
        patch_out = t[:, :-2, :]                        # drop w & CLS

        B,N,D = patch_out.shape        # N=196
        G     = int(math.sqrt(N))      # 14
        x     = patch_out.transpose(1,2).view(B, D, G, G)
        return self.deconv(x)          # [B,1,224,224]


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model  = ViT_GRU_LST(k_hours=6, feature_dim=5).to(device)

for name, p in model.vit.named_parameters():
    if any(s in name for s in ["blocks.10", "blocks.11", "norm"]):
        p.requires_grad = True


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [ ]:
opt       = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()),
                              lr=1e-4, weight_decay=1e-2)
loss_fn   = nn.SmoothL1Loss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    opt, mode='min', factor=0.5, patience=3, verbose=True)


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [ ]:
from tqdm import tqdm, notebook
from pathlib import Path

save_dir = Path("/content/drive/MyDrive/ModelCheckpoints_GRU")
save_dir.mkdir(exist_ok=True)

num_epochs = 10
for ep in range(num_epochs):
    # ---- train ----
    model.train()
    train_loss = 0
    for imgs, w_seq, y in tqdm(train_loader, desc=f"Epoch {ep+1:02d} train"):
        imgs, w_seq, y = imgs.to(device), w_seq.to(device), y.to(device)
        opt.zero_grad()
        pred  = model(imgs, w_seq)
        loss  = loss_fn(pred, y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        opt.step()
        train_loss += loss.item() * imgs.size(0)

    # ---- val ----
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for imgs, w_seq, y in tqdm(val_loader, desc=f"Epoch {ep+1:02d} val  "):
            imgs, w_seq, y = imgs.to(device), w_seq.to(device), y.to(device)
            pred = model(imgs, w_seq)
            val_loss += loss_fn(pred, y).item() * imgs.size(0)

    train_rmse = (train_loss / len(train_loader.dataset))**0.5
    val_rmse   = (val_loss   / len(val_loader.dataset))**0.5
    scheduler.step(val_loss)

    print(f"Epoch {ep+1:02d} ▶ Train {train_rmse:.3f} | Val {val_rmse:.3f}")

    torch.save(model.state_dict(),
        save_dir/f"vit_gru_ep{ep+1:02d}_val{val_rmse:.3f}.pt")


Epoch 01 train:   4%|▍         | 92/2295 [23:40<9:26:44, 15.44s/it] 


KeyboardInterrupt: 